In [ ]:
# # Fitbit Conversational AI - POC Interaction
# 
# This notebook is used to interact with the running `fitbit_agent_service` API.
# It demonstrates the core features required by the assignment, showing how the assistant can provide personalized insights and nudges based on mock user data.
# 
# ### Instructions
# 1. Make sure the FastAPI service is running (`uvicorn fitbit_agent_service:app --reload`).
# 2. Run this notebook cell by cell to see the interaction.

# ## Step 1: Install necessary libraries
# We only need the `requests` library to communicate with our API.
# The `ipython` library is used for better display formatting in the notebook.

In [ ]:
!pip install -q requests ipython

In [7]:
# ## Step 2: Define Helper Functions
# These functions will handle the communication with our API endpoint.

import requests
import json
import uuid
from IPython.display import display, Markdown

# Configuration
BASE_URL = "http://localhost:8000"
SESSION_ID = str(uuid.uuid4())  # Create a single session ID for the conversation
USER_ID = "alex_fitbit"  # Default test user ID

def health_check():
    """Checks if the backend service is running and healthy."""
    display(Markdown("---"))
    display(Markdown("### 1. Checking Service Health..."))
    try:
        response = requests.get(f"{BASE_URL}/health", timeout=5)
        response.raise_for_status()
        status = response.json()
        if status.get("status") == "healthy" and status.get("llm_initialized"):
            display(Markdown("> ✅ **Service is up and running!** The LLM client is initialized."))
            return True
        else:
            display(Markdown("> ❌ **Service has an issue.** Check the server logs."))
            print(status)
            return False
    except requests.exceptions.RequestException as e:
        display(Markdown(f"> ❌ **Service is not reachable.** Please make sure the `uvicorn fitbit_agent_service:app` is running."))
        return False


def run_agent_query(question: str):
    """Sends a question to the agent and displays the conversation."""
    
    # Display the user's query
    display(Markdown(f"**👤 You:**"))
    display(Markdown(f"> {question}"))
    
    try:
        # Create the payload matching the QueryRequest model
        payload = {
            "query": question,
            "session_id": SESSION_ID,
            "user_id": USER_ID
        }
        
        response = requests.post(
            f"{BASE_URL}/query",
            json=payload,
            headers={"Content-Type": "application/json"},
            timeout=30  # Increased timeout to match debug client
        )
        response.raise_for_status()
        
        answer = response.json().get("answer", "I'm not sure how to respond to that.")
        
        # Display the assistant's response
        display(Markdown(f"**🤖 Fitbit Assistant:**"))
        display(Markdown(f"> {answer}"))

    except requests.exceptions.RequestException as e:
        display(Markdown(f"> 🤖 **Error:** Could not get a response from the agent. Details: {e}"))
    
    display(Markdown("---"))



In [8]:
# ## Step 3: Run the Demonstration
# Here, we will simulate a conversation with the assistant, covering the scenarios required by the assignment.

if health_check():
    display(Markdown("### 2. Starting Conversation with Fitbit Assistant..."))
    
    # --- Turn 1: Personalized Insight ---
    # This query asks about sleep, a key piece of personal data.
    run_agent_query("How did I sleep last night?")
    
    # --- Turn 2: Handling a simple user query with data comparison ---
    # This query requires the agent to look at the weekly summary data.
    run_agent_query("Am I more active this week than last week?")

    # --- Turn 3: Follow-up questions or nudges ---
    # This query about steps should trigger a motivational nudge since the user is close to their goal.
    run_agent_query("How am I doing on my steps goal for today?")
    
    # --- Turn 4: A more general wellness question ---
    # This demonstrates the assistant's ability to provide general advice based on the context.
    
    run_agent_query("What can I do to improve my heart rate?")

---

### 1. Checking Service Health...

> ✅ **Service is up and running!** The LLM client is initialized.

### 2. Starting Conversation with Fitbit Assistant...

**👤 You:**

...

**👤 You:**

> What can I do to improve my heart rate?

**🤖 Fitbit Assistant:**

> Great question, Alex! I'm glad you're interested in improving your heart health. Based on your Fitbit data, your resting heart rate today is 65 beats per minute, which is within a healthy range for most adults. However, there are always ways to improve your cardiovascular health. Here are some suggestions:

1. Increase your aerobic exercise: Your data shows you've had 45 minutes in active heart rate zones today, which is fantastic! Try to gradually increase this time to strengthen your heart.

2. Practice stress-reduction techniques: Activities like meditation, deep breathing, or yoga can help lower your resting heart rate over time.

3. Stay hydrated: Proper hydration helps your heart pump blood more efficiently.

4. Get enough sleep: Although your sleep duration was 6.5 hours last night, aim for 7-9 hours to give your heart adequate rest.

5. Maintain a heart-healthy diet: Focus on foods rich in omega-3 fatty acids, whole grains, and plenty of fruits and vegetables.

6. Limit caffeine and alcohol: Both can affect your heart rate, so moderation is key.

Remember, improvements in heart health take time, so be patient and consistent. Keep tracking your progress with your Fitbit, and you'll likely see positive changes over time. Is there a specific area you'd like to focus on first? I'm here to support you on your heart health journey!

---